In [3]:
# ---------------------------------------------------------------------
# 0. Imports & data ----------------------------------------------------
# ---------------------------------------------------------------------
import os
import pandas as pd
import statsmodels.api as sm
import hssm                           # 0.7.0+ recommended
# from your_rl_module import logp_jax_op, model_config   # <- your own
csvfile = os.path.join(os.getcwd(), "hddm2_fixed_final_2states.csv")
dataset = pd.read_csv(csvfile)           # <- path to your file





FileNotFoundError: [Errno 2] No such file or directory: '/Users/jy4657/Desktop/Repositories/HSSM/hddm2_fixed_final_2states.csv'

In [ ]:




# ---------------------------------------------------------------------
# 1. Build centred (and optionally orthogonalised) predictors ----------
# ---------------------------------------------------------------------
# --- (a) z-score "general" uncertainty
dataset["unc_gen_c"] = (
    (dataset["uncertainty_general"] - dataset["uncertainty_general"].mean())
    / dataset["uncertainty_general"].std()
)

# --- (b) z-score "piecemeal" uncertainty
dataset["unc_piece_c"] = (
    (dataset["uncertainty_piecemeal"] - dataset["uncertainty_piecemeal"].mean())
    / dataset["uncertainty_piecemeal"].std()
)

# ------- OPTIONAL: orthogonalise piecemeal vs. general ----------------
# If you set USE_ORTHO = True the piecemeal predictor is residualised
# against the general one, removing collinearity.
USE_ORTHO = False

if USE_ORTHO:
    X = sm.add_constant(dataset["unc_gen_c"])
    resid = sm.OLS(dataset["unc_piece_c"], X).fit().resid
    dataset["unc_piece_c"] = (resid - resid.mean()) / resid.std()

# ---------------------------------------------------------------------
# 2. Define the 't' parameter with separate regression slopes ----------
#    (random intercepts & random slopes for BOTH predictors) ----------
# ---------------------------------------------------------------------
t_param = hssm.Param(
    "t",
    formula = (
        "t ~ 1 + unc_gen_c + unc_piece_c "
        "+ (1 + unc_gen_c + unc_piece_c | participant_id)"
    ),
    prior   = {
        # -------- group (population) means --------
        "Intercept":    hssm.Prior("TruncatedNormal",
                                   lower=0.01, upper=2.0, mu=0.20, sigma=0.10),
        "unc_gen_c":    hssm.Prior("Normal", mu=0.00, sigma=0.15),
        "unc_piece_c":  hssm.Prior("Normal", mu=0.00, sigma=0.15),
        # -------- SDs of the random effects --------
        "Intercept_sd":     hssm.Prior("HalfNormal", sigma=0.20),
        "unc_gen_c_sd":     hssm.Prior("HalfNormal", sigma=0.20),
        "unc_piece_c_sd":   hssm.Prior("HalfNormal", sigma=0.20),
    },
    link = "identity",               # use "softplus" if you need hard positivity
)

# ---------------------------------------------------------------------
# 3. Other parameters (unchanged from your earlier model) --------------
# ---------------------------------------------------------------------
alpha_param = hssm.Param(
    "rl.alpha",
    formula = "rl_alpha ~ 1 + (1 | participant_id)",
    prior   = {"Intercept": hssm.Prior("TruncatedNormal",
                                       lower=0.01, upper=1.0, mu=0.30, sigma=0.10)},
)
scaler_param = hssm.Param(
    "scaler",
    formula = "scaler ~ 1 + (1 | participant_id)",
    prior   = {"Intercept": hssm.Prior("TruncatedNormal",
                                       lower=1.0, upper=4.0, mu=1.50, sigma=0.40)},
)
a_param  = hssm.Param("a",  formula="a ~ 1 + (1 | participant_id)",
    prior={"Intercept": hssm.Prior("TruncatedNormal",
                                   lower=0.30, upper=2.50, mu=1.00, sigma=0.25)})
z_param  = hssm.Param("z",  formula="z ~ 1 + (1 | participant_id)",
    prior={"Intercept": hssm.Prior("TruncatedNormal",
                                   lower=0.10, upper=0.90, mu=0.20, sigma=0.10)})
theta_param = hssm.Param("theta", formula="theta ~ 1 + (1 | participant_id)",
    prior={"Intercept": hssm.Prior("TruncatedNormal",
                                   lower=0.00, upper=1.20, mu=0.30, sigma=0.15)})
w_param = hssm.Param("w",   formula="w ~ 1 + (1 | participant_id)",
    prior={"Intercept": hssm.Prior("TruncatedNormal",
                                   lower=0.10, upper=0.90, mu=0.20, sigma=0.10)})

# ---------------------------------------------------------------------
# 4. Instantiate the hierarchical SSM ---------------------------------
# ---------------------------------------------------------------------
hssm_model = hssm.HSSM(
    data            = dataset,
    model_config    = model_config,
    p_outlier       = 0,
    lapse           = None,
    loglik          = logp_jax_op,            # your custom RL-DDM likelihood
    loglik_kind     = "approx_differentiable",
    noncentered     = True,                   # robust default for few trials/pp
    process_initvals= False,
    include         = [
        alpha_param, scaler_param, a_param, z_param,
        theta_param, w_param,
        t_param,                        # <-- our new spec with two predictors
    ],
)

# ---------------------------------------------------------------------
# 5. Sample ------------------------------------------------------------
# ---------------------------------------------------------------------
idata = hssm_model.sample(
    draws           = 2000,
    chains          = 4,
    target_accept   = 0.9,          # ↑ a bit if you still see divergences
    max_treedepth   = 15,
)

# ---------------------------------------------------------------------
# 6. Posterior utilities ----------------------------------------------
# ---------------------------------------------------------------------
# population-level slopes
beta_gen   = idata.posterior["t_unc_gen_c"]
beta_piece = idata.posterior["t_unc_piece_c"]

# participant-specific slopes (non-centred reconstruction)
beta_gen_i = (
    beta_gen
    + idata.posterior["t_unc_gen_c_offset"]
      * idata.posterior["t_unc_gen_c_sigma"]
)   # dims: chain × draw × participant_id
